**Note.** *The following notebook contains code in addition to text and figures. By default, the code has been hidden. You can click the icon that looks like an eye in the toolbar above to show the code.*

In [ ]:
# Import packages, set preferences, etc.
%matplotlib inline
from brian2 import *
import ipywidgets as ipw
from numpy.random import poisson
import warnings
warnings.filterwarnings("ignore")

prefs.codegen.target = 'weave'
defaultclock.dt = 0.05*ms

In [ ]:
%%html
<!-- hack to improve styling of ipywidgets sliders -->
<style type="text/css">
.widget-hbox .widget-label {
    min-width: 35ex;
    max-width: 35ex;
}
.widget-hslider {
    width: 100%;
}
</style>

# Basic model

We start from the following model of a chopper cell in the ventral cochlear nucleus. The cell is a leaky integrate and fire neuron, defined by the differential equation $$\tau\frac{\mathrm{d}v}{\mathrm{d}t}=-v.$$ If $v>1$ it fires a spike and instantly resets to $v\leftarrow 0$ and is held there for time $t_\mathrm{ref}$. It receives $N$ excitatory and $N$ inhibitory synapses from neurons that fire Poisson spikes at rates $\rho_E$ and $\rho_I$ respectively. The excitatory synapses have weight $w$ and the inhibitory $-w$, so an incoming spike causes an instantaneous change $v\leftarrow v\pm w$. We will sometimes write the inhibitory fraction $\alpha=\rho_I/\rho_E$ and reparametrise the pair $(\rho_E, \rho_I)$ as $(\rho, \alpha)$ where $\rho_E=\rho$ and $\rho_I=\alpha\rho$.

The following interactive figure allows you to visualise the behaviour of this model for different values of the parameters $\tau$, $N$, $w$, $\rho$, $\alpha$, $t_\mathrm{ref}$.

In [ ]:
def basic_model(N=50, tau_ms=10, w=0.02, rho_Hz=200, alpha=0.0, t_ref_ms=0.1):
    # Set parameters
    repeats = 100
    duration = 100*ms
    tau = tau_ms*ms
    rho = rho_Hz*Hz
    t_ref = t_ref_ms*ms
    # Define and run the model
    eqs = '''
    dv/dt = -v/tau : 1 (unless refractory)
    '''
    G = NeuronGroup(100, eqs, threshold='v>1', reset='v=0',
                    refractory=t_ref, method='euler')
    Pe = PoissonInput(G, 'v', N, rho, weight=w)
    if alpha:
        Pi = PoissonInput(G, 'v', N, rho*alpha, weight=-w)
    spikemon = SpikeMonitor(G)
    statemon = StateMonitor(G, 'v', record=[0])
    run(duration)
    # Compute ISI histograms
    isi = []
    for train in spikemon.spike_trains().values():
        train.sort()
        isi.append(diff(train))
    isi = hstack(isi)
    cv = std(isi)/mean(isi)
    # Plot results
    figure(figsize=(10, 2.5))
    subplot(131)
    plot(spikemon.t/ms, spikemon.i, '.k')
    xlabel('Time (ms)')
    ylabel('Repeat number')
    title('Spike raster plot')
    xlim(0, duration/ms)
    ylim(0, repeats)
    
    subplot(132)
    plot(statemon.t/ms, statemon.v.T, '-k')
    xlabel('Time (ms)')
    ylabel('v')
    title('Membrane potential trace')
    xlim(0, duration/ms)
    ylim(-0.2, 1.2)
    axhline(0, ls=':', c='r')
    axhline(1, ls=':', c='g')
    
    subplot(133)
    hist(isi/ms, fc='k', bins=arange(60)*0.5)
    yticks([])
    ylabel('Frequency')
    xlabel('ISI (ms)')
    title('Interspike interval histogram')
    #title('CV = %.2f' % cv)
    text(0.95, 0.9, 'CV = %.2f' % cv, ha='right', va='top',
         bbox=dict(facecolor='white'),
         transform=gca().transAxes)
    tight_layout()

ipw.interact(basic_model,
             N=ipw.IntSlider(
                 min=1, max=100, step=1, value=50,
                 continuous_update=False,
                 description="Number of inputs $N$"),
             tau_ms=ipw.FloatSlider(
                 min=0.1, max=20.0, step=0.1, value=10.0,
                 continuous_update=False,
                 description=r"Membrane time constant $\tau$ (ms)"),
             rho_Hz=ipw.FloatSlider(
                 min=10, max=500, step=10, value=200.0,
                 continuous_update=False,
                 description=r"Excitatory input firing rate $\rho$ (sp/s)"),
             w=ipw.FloatSlider(
                 min=0, max=1, step=0.01, value=0.02,
                 continuous_update=False,
                 description=r"Synaptic weight $w$"),
             alpha=ipw.FloatSlider(
                 min=0, max=1.5, step=0.05, value=0,
                 continuous_update=False,
                 description=r"Inhibitory fraction $\alpha$"),
             t_ref_ms=ipw.FloatSlider(
                 min=0, max=5, step=0.05, value=0.1,
                 continuous_update=False,
                 description=r"Refractory period $t_\mathrm{ref}$ (ms)"),
             );

## Diffusion approximation

Some text explaining diffusion approximation here.

In [ ]:
def diffusion_panel(num_inputs=10, tau_ms=10, rate_Hz=100, duration_ms=200):
    tau = tau_ms*ms
    rate = rate_Hz*Hz
    duration = duration_ms*ms
    weight = 1.0
    mu = weight*rate*num_inputs*tau
    sigma = weight*sqrt(rate*num_inputs*tau)
    nsteps = int(ceil(duration/defaultclock.dt))
    I = TimedArray(weight*poisson(num_inputs*rate*defaultclock.dt, size=nsteps)*tau/defaultclock.dt,
                   dt=defaultclock.dt)
    eqs = '''
    dv/dt = -v/tau : 1
    dw/dt = (mu-w)/tau+sigma*xi*tau**-0.5 : 1
    du/dt = (I(t)-u)/tau : 1
    '''
    P = PoissonGroup(num_inputs, rate)
    G = NeuronGroup(1, eqs, method='euler')
    S = Synapses(P, G, 'dummy:1', on_pre='v += weight')
    S.connect()
    M = StateMonitor(G, ['v', 'w', 'u'], record=True)
    Network(P, G, S, M).run(duration)
    plot(M.t/ms, M.v[0], label="Original")
    plot(M.t/ms, M.w[0], label="Diffusion approximation")
    yticks([])
    xlabel('Time (ms)')
    ylabel('Membrane potential')
    legend(loc='upper left')
    
ipw.interact(diffusion_panel,
             num_inputs=ipw.IntSlider(min=1, max=50, step=1, value=10,
                                      continuous_update=False,
                                      description="Num inputs $N$"),
             tau_ms=ipw.FloatSlider(min=0.1, max=20.0, step=0.1, value=10.0,
                                    continuous_update=False,
                                    description=r"Membrane time constant $\tau$ (ms)"),
             rate_Hz=ipw.FloatSlider(min=10, max=500, step=10, value=100.0,
                                     continuous_update=False,
                                     description=r"Input firing rate $\rho$ (sp/s)"),
             duration_ms=ipw.FloatSlider(min=50, max=500, step=50, value=200,
                                         continuous_update=False,
                                         description=r"Duration (ms)"),
             );